## <span style="color:red">Decorators</span>

Un decorator in Python è una funzione (più in generale si può dire un **oggetto chiamabile**) che viene usato per modificare (decorare) il comportamento di una funzione o di una classe (e dei suoi oggetti).

Il decoratore prende in input la funzione (o la classe) e restituisce la funzione (o la classe) modificata

Abbiamo già utilizzato questa struttura (senza averla chiamata col nome appropriato) quando abbiamo usato @property, che è un esempio notevole di decoratore

Ora vediamo la nozione in modo più approfondito

### Ricordiamo dapprima che le funzioni sono first-class object e, a tal proposito, vediamo qualche esempio

In [25]:
# Funzione che calcola i numeri di Fibonacci fino all'n-esimo
def Fibonacci(n):
    f0 = 0
    f1 = 1
    for _ in range(n-1):
      f0, f1 = f1, f0+f1
    return f1

In [ ]:
Fibonacci(100)  # L'implementazione è efficiente perché... si tiene lontana 
                # dalla "tentazione" della ricorsione

### Una funzione (si badi, non il risultato della chiamata di una funzione) può essere "assegnata" ad un nome

In [ ]:
fibo = Fibonacci
fibo(100)          # fibo e fibonacci sono la stessa funzione

### Una funzione può essere passata come parametro ad un'altra funzione

In [ ]:
def tabulate(sequence, f):           # Type checking (con possibili errori)
    return [f(x) for x in sequence]  # solo a run-time, come ben sappiamo

In [ ]:
import math
tabulate([x*0.01 for x in range(157)], math.sin)  # Qui tutto OK. Calcola il seno da 0 a pi/2

In [ ]:
tabulate(['foo','bar','baz'],math.sin)  # Qui ovviamente no!

### Una funzione può anche essere "restituita" come risultato di un'altra funzione

In [ ]:
def maketab(start, step, npoints):
    '''Restituisce un "tabulatore" in cui sono fissati i punti in cui
    la funzione (ogni fuzione) verrà calcolata'''
    from math import pi
    sequence = [start+pi*i*step for i in range(npoints)] # Il passo è implicitamente espresso come multiplo di pi
    def tabulate(f):
        return [f(x) for x in sequence]
    return tabulate

In [ ]:
calcola = maketab(0,0.25,9) # Da 0 a 2pi con passo pi/4

In [ ]:
calcola(math.sin)   # Calcola il seno da 0 a 2pi con passo pi/4

In [ ]:
calcola(math.cos)  # Calcola il coseno da 0 a 2pi con passo pi/4  

#### Tecnicamente *calcola* è un oggetto che si chiama <span style="color:red">closure</span> (cioè <span style="color:red">chiusura</span>) e non semplicamente una funzione

Ci chiediamo infatti: da dove *calcola* prende i punti cui applicare la funzione? 

La risposta sembra ovvia: i punti sono il valore di *sequence*.  un nome locale a *maketab*. 

Mmmm, ... non così ovvio! Il nome *sequence* è infatti locale per *maketab* ma non locale per *tabulate* (e dunque per *calcola*). 

Questo vuol forse dire che *maketab* viene comunque (in quache modo) "chiamata" quando *calcola* viene invocata? 

In [ ]:
del(maketab)            # Ora maketab non esiste più
calcola(lambda x: x**2) # ... ma calcola funziona egualmente

Decisamente no. *maketab* non viene più chiamata? Allora?

La risposta sta appunto nel concetto di *chiusura*. Ciò che *maketab* ha restituito è una funzione che "ricorda" i valori nello scope (non locale) in cui è stata definita, anche se questi non sono più presenti in memoria. Una *chiusura* è cioè una funzione con un namespace (di nomi non locali) "attaccato"

#### Ancora un esempio

In [ ]:
def makePoly(*coefficients):
    def eval(x):
        value = 0.0
        for c in reversed(coefficients): # coenfficient è non locale per eval
            value = value*x + c
        return value
    return eval

In [ ]:
p = makePoly(1,-2,1)   # x**2-2x+1 = (x-1)**2

In [ ]:
for i in range(5):
    print(p(i))

### Un <span style="color:red">decoratore</span> mette insieme le due "costruzioni": funzione come parametro e funzione come valore di ritorno, introducendo anche una <u>sintassi specifica</u>

In [12]:
import math

In [13]:
def g(x):
    '''g è una semplice funzione che, dato x, stampa x e il valore di sin(x)'''
    print(x,'\t',math.sin(x))

In [14]:
g(0.2)

0.2 	 0.19866933079506122


#### Vogliamo ora "abbellire" l'output facendo in modo che qualsiasi sia la funzione (di una variabile) la stampa sia preceduta da un'intestazione

In [15]:
def decoratore(f):
    def decorata(x):
        print('x\t f(x)')
        return f(x)
    return decorata

In [16]:
g = decoratore(g)

In [20]:
g(0.2)

x	 f(x)
0.2 	 0.19866933079506122


#### Naturalmente non poteva mancare lo "zucchero sintattico"

In [11]:
@decoratore
def g(x):
    print(x,'\t',math.sin(x))

In [8]:
g(0.2)

x	 f(x)
0.2 	 0.19866933079506122


In [9]:
def decoratore(f):
    def decorata(*x):
        print('x\t f(x)')
        for z in x:
            f(z)
    return decorata

In [12]:
g(0.01,0.02,0.03)

x	 f(x)
0.01 	 0.009999833334166664
0.02 	 0.01999866669333308
0.03 	 0.02999550020249566


In [14]:
g(*[i/10 for i in range(10)])

x	 f(x)
0.0 	 0.0
0.1 	 0.09983341664682815
0.2 	 0.19866933079506122
0.3 	 0.29552020666133955
0.4 	 0.3894183423086505
0.5 	 0.479425538604203
0.6 	 0.5646424733950354
0.7 	 0.644217687237691
0.8 	 0.7173560908995228
0.9 	 0.7833269096274834


### Tipici casi d'uso dei decoratori

La decorazione può utilmente "occuparsi" (in particolare, per la chiarezza del codice) dei casi particolari di input che devono essere controllati da un algoritmo.

In [21]:
def roots2(a,b,c):
    from cmath import sqrt
    delta = math.sqrt(b**2-4*a*c)
    x1 = (-b-delta)/(2*a)
    x2 = (-b+delta)/(2*a)
    return x1,x2

In [24]:
from cmath import sqrt
sqrt(-2)

1.4142135623730951j

In [25]:
roots2(0,-2,1)

ZeroDivisionError: float division by zero

In [26]:
def special_cases(f):
    '''Tratta il caso in cui il coefficiente del termine quadratico è nullo'''
    def checker(a,b,c):
        if a == 0:
            assert (b!=0), "Almeno uno fra a e b deve essere non zero"
            return -c/b
        else:
            return f(a,b,c)
    return checker

In [27]:
@special_cases
def roots2(a,b,c):
    from cmath import sqrt
    delta = sqrt(b**2-4*a*c)
    x1 = (-b-delta)/(2*a)
    x2 = (-b+delta)/(2*a)
    return x1,x2

In [32]:
roots2(1,0,2)

(-1.4142135623730951j, 1.4142135623730951j)

In [31]:
roots2(0,3,1)

-0.3333333333333333

In [33]:
roots2(0,0,1)

AssertionError: Almeno uno fra a e b deve essere non zero

Un secondo caso di utilizzo è la raccolta di statistiche sull'uso di una funzione (tempo di esecuzione, numero di volte che è stata chiamata, ...)

In [2]:
from time import time

In [23]:
time()

1572260584.2236798

In [15]:
n = 35

In [3]:
def Fibonacci_ric(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_ric(n-2)+Fibonacci_ric(n-1)

In [19]:
start_time = int(round(time() * 1000)) #tempo in millisecondi (trascorso a partire dal 1/1/1970
Fibonacci_ric(n)
stop_time = int(round(time() * 1000))
elapsed_time = stop_time-start_time
print("Tempo di esecuzione di Fibonacci_ric({}): {}ms".format(n, elapsed_time))

Tempo di esecuzione di Fibonacci_ric(30): 1949ms


In [25]:
def my_timer(f):
    from time import time
    def timedfun(n):
        start_time = int(round(time() * 1000))
        res = f(n)
        stop_time = int(round(time() * 1000))
        timedfun.time = stop_time-start_time
        return res
    return timedfun

@my_timer
def Fibonacci_ric(n):
    if n == 0 or n == 1:
        return n
    return Fibonacci_ric(n-2)+Fibonacci_ric(n-1)

@my_timer
def Fibonacci(n):
    f0 = 0
    f1 = 1
    for _ in range(n-1):
      f0, f1 = f1, f0+f1
    return f1

In [ ]:
n = 35
res = Fibonacci_ric(n)
print("Tempo di esecuzione di Fibonacci_ric({}): {}ms".format(n, Fibonacci_ric.time))
print("F_{}={}".format(n,res))
res = Fibonacci(n)
print("Tempo di esecuzione di Fibonacci({}): {}ms".format(n, Fibonacci.time))
print("F_{}={}".format(n,res))

#### Piccola curiosità su Fibonacci

In [40]:
from math import sqrt
phi = (1+sqrt(5))/2    # Phi è il rapporto aureo
F0 = Fibonacci(1)
print("n\tRapporto F_n/F_n-1={}\tScarto con phi")
for n in range(2,51):
    F1 = Fibonacci(n)
    R = F1/F0
    F0 = F1
    print("{0}\t{1:.10f}\t\t{2:.3}".format(n,R,(R-phi)/phi))

n	Rapporto F_n/F_n-1={}	Scarto con phi
2	1.0000000000		-0.382
3	2.0000000000		0.236
4	1.5000000000		-0.0729
5	1.6666666667		0.0301
6	1.6000000000		-0.0111
7	1.6250000000		0.00431
8	1.6153846154		-0.00164
9	1.6190476190		0.000626
10	1.6176470588		-0.000239
11	1.6181818182		9.14e-05
12	1.6179775281		-3.49e-05
13	1.6180555556		1.33e-05
14	1.6180257511		-5.09e-06
15	1.6180371353		1.94e-06
16	1.6180327869		-7.43e-07
17	1.6180344478		2.84e-07
18	1.6180338134		-1.08e-07
19	1.6180340557		4.14e-08
20	1.6180339632		-1.58e-08
21	1.6180339985		6.04e-09
22	1.6180339850		-2.31e-09
23	1.6180339902		8.81e-10
24	1.6180339882		-3.37e-10
25	1.6180339890		1.29e-10
26	1.6180339887		-4.91e-11
27	1.6180339888		1.88e-11
28	1.6180339887		-7.16e-12
29	1.6180339888		2.74e-12
30	1.6180339887		-1.05e-12
31	1.6180339888		3.99e-13
32	1.6180339887		-1.52e-13
33	1.6180339887		5.82e-14
34	1.6180339887		-2.22e-14
35	1.6180339887		8.51e-15
36	1.6180339887		-3.29e-15
37	1.6180339887		1.24e-15
38	1.6180339887		-5.49e-16
39

#### I risultati mostrano che $F_n$ asintoticamente si comporta come $\phi^n$ 

### Tempo di esecuzione di *insertion sort*

In [41]:
@my_timer
def insSort(A):
    n = len(A)
    for i in range(1,n):
        temp = A[i]
        j = i-1
        while j>=0 and A[j]>temp:
            A[j+1] = A[j]
            j = j-1
        A[j+1] = temp

In [42]:
from random import randint

In [45]:
n = 40000
A = [randint(1,100) for _ in range(n)]

In [46]:
insSort(A)
print("Tempo di esecuzione di insSort(A): {}ms".format(insSort.time))

Tempo di esecuzione di insSort(A): 37573ms


### Un timer per funzioni con generico numero di parametri

In [57]:
def general_timer(f):
    def timedfun(*args, **kw):
        start_time = int(round(time() * 1000))
        res = f(*args, **kw)
        stop_time = int(round(time() * 1000))
        timedfun.time = stop_time-start_time
        return res
    return timedfun

In [50]:
@general_timer
def insSort(A):
    n = len(A)
    for i in range(1,n):
        temp = A[i]
        j = i-1
        while j>=0 and A[j]>temp:
            A[j+1] = A[j]
            j = j-1
        A[j+1] = temp

In [53]:
insSort(A)
print("Tempo di esecuzione di insSort(A): {}ms".format(insSort.time))

Tempo di esecuzione di insSort(A): 5ms


### Anziché il tempo, si potrebbe voler misurare (ad esempio) il numero di chiamate di una funzione ricorsiva

In [34]:
def callcnt(f):
    def counter(*args, **kw):
        counter.calls += 1
        return f(*args, **kw)
    return counter

In [35]:
@callcnt
def Fibonacci_rec(n):
    if n == 1 or n == 2:
        return 1
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [36]:
n = 25
Fibonacci_rec.calls = 0
v = Fibonacci_rec(n)
print(f"Il numero C({n}) di chiamate ricorsive è {Fibonacci_rec.calls}")
print(f"Il valore di f({n}) è {v}")
print(f"Controllo, C({n}) == 2*f({n})-1: {Fibonacci_rec.calls==2*v-1}")

Il numero C(25) di chiamate ricorsive è 150049
Il valore di f(25) è 75025
Controllo, C(25) == 2*f(25)-1: True


### Qualche primo elemento di *introspection*

#### La <span style="color:red">introspezione</span>  è la capacità di un oggetto di avere, a tempo di esecuzione, conoscenza degli attributi propri e di altri oggetti

In [41]:
def una_funzione(*x):
    pass

def valuta(f,*x):
    print(f"Sto valutando la funzione {f.__name__}")
    return f(*x)
valuta(una_funzione)

Sto valutando la funzione una_funzione


Con i decoratori di mezzo ci possono però essere problemi

In [43]:
print(Fibonacci_rec.__name__)

counter


In [47]:
def callcnt(f):
    from functools import wraps
    @wraps(f)
    def counter(*args, **kw):
        counter.calls += 1
        return f(*args, **kw)
    return counter

In [48]:
@callcnt
def Fibonacci_rec(n):
    if n == 1 or n == 2:
        return 1
    return Fibonacci_rec(n-2)+Fibonacci_rec(n-1)

In [49]:
print(Fibonacci_rec.__name__)

Fibonacci_rec


#### Per concludere: un utile *template* generale

In [ ]:
def decoratore(f):
    from functools import wrap
    @wrap(f)
    def modificata(*args,**kwars):
        # Eventuale codice da eseguire prima di f
        ris = f(*args,**kwargs) # Esecuzione di f e memorizzazione del risultato
        # Eventuale codice da eseguire dopo f
        return ris
    return modificata

### Digressione. \__call\__: un interessante "metodo magico"

Vogliamo implementare un semplice generatore pseudo-casuale, da non utilizzare per scopi crittografici....

Il generatore produce una sequenza di numeri $<R_n>$ nell'intervallo \[0,1), così definiti:

$$\begin{eqnarray*}
X_n&=&a\cdot X_{n-1} \mathrm{mod}\ m\\
R_n&=&\frac{X_n}{m}
\end{eqnarray*}
$$
con $n=1,2,\ldots$, dove $a, m$ e $X_0$ sono opportunamente scelti

In [52]:
class myrand:
    '''Un semplice generatore pseudo-casuale'''
    def __init__(self):
        from time import time
        self._a = 16807               # a = 16807
        self._m = (1 << 31) - 1       # m = 2^31 -1
        self._d = 1.0/self._m
        self._x = int(time())%self._m # x_0 "casuale", dipendente dal tempo
        if self._x==0:
            self._x = 1               # ... ma non 0
        for i in range(10000):  # Facciamo un po' di giri "a vuoto" per rendere i numeri
                                # (apparentemente) indipendenti dal seme iniziale
            self.__call__()
    
    def __call__(self):
        self._x = (self._a*self._x)%self._m
        return self._x*self._d

In [53]:
r = myrand()

In [55]:
for _ in range(10):
    print(r())   # r() è zucchero sintattico per r.__call__()

0.41678541918135503
0.9125401810335648
0.0628226311238588
0.8599612986948161
0.3695471637740485
0.9791815504334781
0.10431813546657476
0.27490278672189583
0.2911364349029662
0.1300614141533437


L'esempio (semplice) illustra come il metodo \__call\__ possa essere chiamato col nome stesso della classe; questo ne rende l'uso di comprensione immediata. Si tratta di un  possibile modo per realizzare "funzioni con stato".